In [ ]:
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet



7[https://d37ci6vzurychx.cloudfr]87[Files: 0  Bytes: 0  [0 B/s] Re]87Saving 'yellow_tripdata_2024-10.parquet.2'
87yellow_tripdata_2024   1% [>                             ]    1.16M    --.-KB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87yellow_tripdata_2024   9% [=>                            ]    5.91M    4.74MB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87yellow_tripdata_2024  19% [====>                         ]   12.02M    5.42MB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87yellow_tripdata_2024  29% [=======>                      ]   17.95M    5.59MB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87yellow_tripdata_2024  36% [==========>                   ]   22.57M    5.34MB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87yellow_tripdata_2024  43% [============>                 ]   26.64M    5.09MB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87yellow_tripdata_2024  49% [=============>                ]   30.41M    4.87MB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87yellow_tripdata_2024  59% [================>

## Question 1: Install Spark and PySpark

    - Install Spark
    - Run PySpark
    - Create a local spark session
    - Execute spark.version.

What's the output?

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/13 17:58:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.version

'3.5.5'

25/03/13 17:59:02 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Question 2: Yellow October 2024
Read the October 2024 Yellow into a Spark Dataframe.

Repartition the Dataframe to 4 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

In [48]:
df = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [35]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [36]:
df

DataFrame[VendorID: int, tpep_pickup_datetime: timestamp_ntz, tpep_dropoff_datetime: timestamp_ntz, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double]

In [14]:
df = df.repartition(4)
df.write.parquet('yellow/2024/10/')

In [49]:
!ls -lh ./yellow/2024/10

total 90M
-rw-r--r--. 1 dauntless dauntless 23M Mar 13 18:15 part-00000-2a68f72a-870f-4ce2-a040-79c5209a1836-c000.snappy.parquet
-rw-r--r--. 1 dauntless dauntless 23M Mar 13 18:15 part-00001-2a68f72a-870f-4ce2-a040-79c5209a1836-c000.snappy.parquet
-rw-r--r--. 1 dauntless dauntless 23M Mar 13 18:15 part-00002-2a68f72a-870f-4ce2-a040-79c5209a1836-c000.snappy.parquet
-rw-r--r--. 1 dauntless dauntless 23M Mar 13 18:15 part-00003-2a68f72a-870f-4ce2-a040-79c5209a1836-c000.snappy.parquet
-rw-r--r--. 1 dauntless dauntless   0 Mar 13 18:15 _SUCCESS


## Question 3: Count records
How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

In [81]:
from pyspark.sql import functions as F

df \
    .withColumn('tpep_pickup_datetime', F.to_date(df.tpep_pickup_datetime)) \
    .filter("tpep_pickup_datetime = '2024-10-15'") \
    .count()

128893

In [60]:
df.createOrReplaceTempView('trips_data')

In [82]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    trips_data
WHERE
    to_date(tpep_pickup_datetime) = '2024-10-15';
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [83]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

## Question 4: Longest trip
What is the length of the longest trip in the dataset in hours?

In [107]:
spark.sql("""
SELECT
    (unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 3600 AS trip_duration_hours
FROM 
    trips_data
ORDER BY
    1 DESC
LIMIT 10;
""").show()

+-------------------+
|trip_duration_hours|
+-------------------+
| 162.61777777777777|
|            143.325|
| 137.76055555555556|
| 114.83472222222223|
|  89.89833333333333|
|  89.44611111111111|
|  70.29916666666666|
|  67.57333333333334|
|  66.06666666666666|
|            46.4225|
+-------------------+



## Question 6: Least frequent pickup location zone

In [108]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv



7[https://d37ci6vzurychx.cloudfr]87[Files: 0  Bytes: 0  [0 B/s] Re]87Saving 'taxi_zone_lookup.csv'
87taxi_zone_lookup.csv 100% [=============================>]    2.98K    --.-KB/s87HTTP response 200  [https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv]
87taxi_zone_lookup.csv 100% [=============================>]    2.98K    --.-KB/s87[Files: 1  Bytes: 2.98K [2.74KB]8

In [119]:
df_taxi_zones = spark.read.option("header", "true").csv('taxi_zone_lookup.csv')

In [129]:
df_taxi_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [112]:
df_taxi_zones.createOrReplaceTempView('taxi_zones')

In [122]:
df_result = df.join(df_taxi_zones, df.PULocationID == df_taxi_zones.LocationID)

In [128]:
df_result.createOrReplaceTempView('trips_and_zones')

In [156]:
spark.sql("""
SELECT
    Zone, COUNT(1)
FROM 
    trips_and_zones
GROUP BY
    1
ORDER BY
    2 ASC
LIMIT 10;
""").show(truncate=False)

+---------------------------------------------+--------+
|Zone                                         |count(1)|
+---------------------------------------------+--------+
|Governor's Island/Ellis Island/Liberty Island|1       |
|Rikers Island                                |2       |
|Arden Heights                                |2       |
|Jamaica Bay                                  |3       |
|Green-Wood Cemetery                          |3       |
|Charleston/Tottenville                       |4       |
|Rossville/Woodrow                            |4       |
|West Brighton                                |4       |
|Port Richmond                                |4       |
|Eltingville/Annadale/Prince's Bay            |4       |
+---------------------------------------------+--------+



In [132]:
!ls

download_data.sh      yellow
Homework.ipynb	      yellow_tripdata_2024-10.parquet
README.md	      yellow_tripdata_2024-10.parquet.1
taxi_zone_lookup.csv  yellow_tripdata_2024-10.parquet.2
